In [ ]:
import re
from os import WEXITED
def get_products(content):
  products = []
  soup = BeautifulSoup(content, "html.parser")
  section = soup.find('section', class_="products-section")
  if not section:
      print("Section 'products-section' not found.")
      return []
  articles = section.find_all('article')
  for article in articles:
    # Link
    link = article.find('a', class_="product-card__heading-link")['href']
    #Značka
    brand = article.find('span', class_="product-card__heading-producer").text.strip()
    # Název produktu
    name = article.find('span', class_="product-card__heading-model").text.strip()
    # Kategorie
    category  = article.find('span', class_="product-card__heading-type").text.strip()

    # Váha
    weight_element = article.find('dd', class_="product-card__parameters--value")
    weight_raw = weight_element.text.strip() if weight_element else None

    weight_value = None
    weight_unit = None
    if weight_raw:
        # Use regex to find number and optional unit (e.g., '420 g', '550g')
        match = re.match(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+)?', weight_raw)
        if match:
            try:
                weight_value = float(match.group(1))
            except ValueError:
                weight_value = None # Assign None if conversion fails
            weight_unit = match.group(2) if match.group(2) else None # Unit might be

    # Aktuální cena
    price_raw = article.find('span', class_="card-price__full").text.strip()
    price_cleaned = price_raw.replace('od ', '').replace('Kč', '').replace('\xa0', '').strip()
    try:
        current_price = float(price_cleaned.replace(',', '.')) # Handle comma as decimal separator if any
    except ValueError:
        current_price = 'None' # Assign None if conversion fails

    # Original cena
    original_price_element = article.find('span', class_="card-price__discount")
    original_price = None
    if original_price_element:
        original_price_raw = original_price_element.text.strip()
        original_price_cleaned = original_price_raw.replace('od ', '').replace('Kč', '').replace('\xa0', '').strip()
        try:
            original_price = float(original_price_cleaned.replace(',', '.')) # Handle comma as decimal separator if any
        except ValueError:
            original_price = 'None' # Assign None if conversion fails

    products.append(
        {
          'link': link,
          'brand': brand,
          'name': name,
          'category': category,
          'weight_value': weight_value,
          'weight_unit': weight_unit,
          'current_price': current_price,
          'original_price': original_price

          }
      )

  return products

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import re # Make sure 're' is imported if not already in the environment

all_products = []

urls = []
for page in range(1, 5):
    urls.append(
        f"https://www.4camping.cz/c/horolezecke-vybaveni/lezecky/?p={page}"
    )
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36'}

for url in urls:
  response = requests.get(url,headers=headers)
  products = get_products(response.content)
  all_products += products
  sleep(5)
  print(f'{url} has been done!')


print('Products have been downloaded')

print(all_products)

https://www.4camping.cz/c/horolezecke-vybaveni/lezecky/?p=1 has been done!
https://www.4camping.cz/c/horolezecke-vybaveni/lezecky/?p=2 has been done!
https://www.4camping.cz/c/horolezecke-vybaveni/lezecky/?p=3 has been done!
https://www.4camping.cz/c/horolezecke-vybaveni/lezecky/?p=4 has been done!
Products have been downloaded
[{'link': '/p/lezecky-ocun-advancer-lu/#37-5-modra-seda', 'brand': 'Ocún', 'name': 'Advancer Lu 2024', 'category': 'Lezečky', 'weight_value': 420.0, 'weight_unit': 'g', 'current_price': 1899.0, 'original_price': 2190.0}, {'link': '/p/lezecky-ocun-sigma/#37-5-modra-oranzova', 'brand': 'Ocún', 'name': 'Sigma', 'category': 'Lezečky', 'weight_value': None, 'weight_unit': None, 'current_price': 2749.0, 'original_price': 3590.0}, {'link': '/p/damske-lezecky-evolv-shaman-lv/#37-5-cervena-modra', 'brand': 'Evolv', 'name': 'Shaman LV', 'category': 'Dámské lezečky', 'weight_value': None, 'weight_unit': None, 'current_price': 3825.0, 'original_price': 4250.0}, {'link': '/p

In [ ]:
import pandas as pd

df_products = pd.DataFrame(all_products)

# Save the DataFrame to a CSV file
df_products.to_csv('products_data.csv', index=False, encoding='utf-8')

print("Data byla úspěšně uložena do 'products_data.csv'.")

Data byla úspěšně uložena do 'products_data.csv'.
